In [8]:
# from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pymongo
# import datetime
# import time
import pandas as pd
import numpy as np
# import re
import warnings
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
import string
%matplotlib inline
import matplotlib.pyplot as plt

In [9]:
mc = pymongo.MongoClient()
db = mc['chordify']
raw_html = db['raw_html']
parsed_songs_db = db["parsed_songs"]

In [10]:
html_docs = list(raw_html.find())

In [67]:
raw_html.find().count()

6261

In [12]:
parsed_songs = list(parsed_songs_db.find())

In [79]:
parsed_songs_db.find().count()

4130

In [13]:
def get_phrase_for_chord(one_parsed_song):
    word_list = one_parsed_song['words']
    chord_phrase_tuples = []
    for chord_set in one_parsed_song['chord_idxs']:
        word_idx = chord_set[0]
        chord_name = chord_set[1]
        phrase = ' '.join(word_list[(word_idx - 8):(word_idx + 3)])
        chord_phrase_tuples.append((chord_name, phrase))
    return chord_phrase_tuples

In [58]:
get_phrase_for_chord(parsed_songs[6])

[('E', ''),
 ('A', ''),
 ('E', ''),
 ('A', ''),
 ('A', "Don't waste your words they're just lies I cried she was"),
 ('E', 'just lies I cried she was deaf And she worked on'),
 ('A', 'lies I cried she was deaf And she worked on my'),
 ('E', 'she was deaf And she worked on my face until breaking'),
 ('A', 'was deaf And she worked on my face until breaking my'),
 ('A', 'on my face until breaking my eyes Then said What else'),
 ('B7', 'said What else you got left It was then that I'),
 ('E', "up to leave But she said Don't forget Everybody must give"),
 ('A', "leave But she said Don't forget Everybody must give something back"),
 ('A',
  "Don't forget Everybody must give something back For something they get"),
 ('E', 'must give something back For something they get I stood there'),
 ('E', 'they get I stood there and hummed I tapped on her'),
 ('A', 'and hummed I tapped on her drum and asked her how'),
 ('A', 'on her drum and asked her how come And she buttoned'),
 ('E', 'drum and asked h

In [15]:
def get_all_phrase_chord_tuples(parsed_songs):
    all_chord_phrase_tuples = []
    for song in parsed_songs:
        chord_phrase_tuples = get_phrase_for_chord(song)
        all_chord_phrase_tuples.extend(chord_phrase_tuples)
    return all_chord_phrase_tuples

In [21]:
pct = get_all_phrase_chord_tuples(parsed_songs[5:8])

In [56]:
def make_phrase_is_minor_list(chord_phrase_tuples):
    phrases = []
    chords = []
    is_minor = []
    for chord_phrase_tup in chord_phrase_tuples:
        phrases.append(chord_phrase_tup[1])
        chords.append(chord_phrase_tup[0])
        is_minor.append('m' in chord_phrase_tup[0])
    return phrases, chords, is_minor

good_chords = ['A', 'C', 'D', 'G']

def get_list_of_chords(chord_phrase_tuples):
    is_A = []
    is_C = []
    is_D = []
    is_G = []
    other_chord = []
    for tup in chord_phrase_tuples:
        is_A.append(tup[0] == 'A')
        is_C.append(tup[0] == 'C')
        is_D.append(tup[0] == 'D')
        is_G.append(tup[0] == 'G')
        other_chord.append(tup[0] not in good_chords)
    return is_A, is_C, is_D, is_G, other_chord

def make_phrase_chord_df(phrases, chords, is_minor, is_A, is_C, is_D, is_G, other_chord):
    df = pd.DataFrame({
        'chords': chords,
        'A': is_A,
        'C': is_C,
        'D': is_D,
        'G': is_G,
        'other_chord': other_chord,
        'is_minor': is_minor,
        'words': phrases
    })
    return df


In [53]:
is_A, is_C, is_D, is_G, other_chord = get_list_of_chords(pct)

In [54]:
phrases, chords, is_minor = make_phrase_is_minor_list(pct)

In [57]:
make_phrase_chord_df(phrases, chords, is_minor, is_A, is_C, is_D, is_G, other_chord)

,A,C,D,G,chord,is_minor,other_chord,words
0,False,False,False,True,G,False,False,
1,False,True,False,False,C,False,False,
2,False,True,False,False,C,False,False,to the doctor he says I'm alright I knew he was
3,False,True,False,False,C,False,False,knew he was lyin' I'm losing my sight He shoul...
4,False,False,False,True,G,False,False,should have examined the eyes of my mind 20 20...
5,False,False,True,False,D,False,False,eyes of my mind 20 20 vision and walkin' round...
6,False,False,False,False,E,False,True,
7,True,False,False,False,A,False,False,
8,False,False,False,False,E,False,True,
9,True,False,False,False,A,False,False,


In [72]:
def get_all_phrase_chord_tuples(parsed_songs):
    chord_phrase_tuples = []
    for song in parsed_songs:
        one_song_chord_phrase_tuples = get_phrase_for_chord(song)
        chord_phrase_tuples.extend(one_song_chord_phrase_tuples)
    return chord_phrase_tuples


def make_phrase_is_minor_list(chord_phrase_tuples):
    phrases = []
    chords = []
    is_minor = []
    for chord_phrase_tup in chord_phrase_tuples:
        phrases.append(chord_phrase_tup[1])
        chords.append(chord_phrase_tup[0])
        is_minor.append('m' in chord_phrase_tup[0])
    return phrases, chords, is_minor

good_chords = ['A', 'C', 'D', 'G']

def get_list_of_chords(chord_phrase_tuples):
    is_A = []
    is_C = []
    is_D = []
    is_G = []
    other_chord = []
    for tup in chord_phrase_tuples:
        is_A.append(tup[0] == 'A')
        is_C.append(tup[0] == 'C')
        is_D.append(tup[0] == 'D')
        is_G.append(tup[0] == 'G')
        other_chord.append(tup[0] not in good_chords)
    return is_A, is_C, is_D, is_G, other_chord


def make_phrase_chord_df(chord_phrase_tuples):
    phrases, chords, is_minor = make_phrase_is_minor_list(chord_phrase_tuples)
    is_A, is_C, is_D, is_G, other_chord = get_list_of_chords(chord_phrase_tuples)
    df = pd.DataFrame({
        'chords': chords,
        'A': is_A,
        'C': is_C,
        'D': is_D,
        'G': is_G,
        'other_chord': other_chord,
        'is_minor': is_minor,
        'words': phrases
    })
    return df

In [73]:
pct_more = get_all_phrase_chord_tuples(parsed_songs[10:15])

In [75]:
make_phrase_chord_df(pct_more)

,A,C,D,G,chords,is_minor,other_chord,words
0,False,False,False,False,Dm,True,True,
1,False,True,False,False,C,False,False,
2,False,False,False,False,Dm,True,True,I went out one morning to breathe the air arou...
3,False,True,False,False,C,False,False,went out one morning to breathe the air around...
4,False,False,False,False,Dm,True,True,to breathe the air around Tom Paine's I spied ...
5,False,True,False,False,C,False,False,air around Tom Paine's I spied the fairest dam...
6,False,False,False,False,Dm,True,True,Tom Paine's I spied the fairest damsel that ev...
7,False,True,False,False,C,False,False,I spied the fairest damsel that ever did walk ...
8,False,False,False,False,F,False,True,damsel that ever did walk in chains I offer'd ...
9,False,False,False,False,Am,True,True,did walk in chains I offer'd her my hand she took


In [76]:
parsed_songs[10]

{'_id': ObjectId('5afb4d8786b136c8b4be5bbc'),
 'chord_idxs': [[0, 'Dm'],
  [5, 'C'],
  [9, 'Dm'],
  [10, 'C'],
  [14, 'Dm'],
  [17, 'C'],
  [19, 'Dm'],
  [21, 'C'],
  [25, 'F'],
  [28, 'Am'],
  [30, 'Dm'],
  [31, 'C'],
  [36, 'Dm'],
  [39, 'C'],
  [41, 'Dm'],
  [43, 'C'],
  [44, 'Dm'],
  [46, 'Dm'],
  [49, 'C'],
  [52, 'Dm'],
  [54, 'C'],
  [57, 'Dm'],
  [62, 'C'],
  [64, 'Dm'],
  [68, 'C'],
  [71, 'F'],
  [75, 'Am'],
  [77, 'Dm'],
  [80, 'C'],
  [83, 'Dm'],
  [86, 'C'],
  [89, 'Dm'],
  [90, 'C'],
  [93, 'Dm'],
  [97, 'C'],
  [99, 'Dm'],
  [101, 'C'],
  [104, 'Dm'],
  [107, 'C'],
  [110, 'Dm'],
  [111, 'C'],
  [114, 'F'],
  [120, 'Am'],
  [122, 'Dm'],
  [124, 'C'],
  [125, 'Dm'],
  [127, 'Dm'],
  [131, 'C'],
  [134, 'Dm'],
  [136, 'C']],
 'words': ['As',
  'I',
  'went',
  'out',
  'one',
  'morning',
  'to',
  'breathe',
  'the',
  'air',
  'around',
  'Tom',
  "Paine's",
  'I',
  'spied',
  'the',
  'fairest',
  'damsel',
  'that',
  'ever',
  'did',
  'walk',
  'in',
  'chains',
  '